In [4]:
# pip install seaborn

In [5]:
%matplotlib qt5
import pyaccel
import pymodels
import numpy as np
import matplotlib.pyplot as plt
import scipy.integrate as scyint
import pandas as pd
from mathphys import constants
from mathphys.functions import save_pickle, load_pickle
from mathphys.beam_optics import beam_rigidity

import touschek_pack.functions as tousfunc
import touschek_pack.Classes as tousclass


In [6]:
acc = pymodels.si.create_accelerator()
acc = pymodels.si.fitted_models.vertical_dispersion_and_coupling(acc)

tousan = tousclass.Tous_analysis(acc)
spos = pyaccel.lattice.find_spos(acc, indices='closed')


In [7]:
# # indices = pyaccel.lattice.find_indices(acc, 'fam_name', 'B2')
# fam_data = pymodels.si.get_family_data(acc)

# s = spos[10:30]
# tousan.plot_scat_table(s, n_c=1, n_r=1)

# # fam_data['QN']['index']


In [8]:
# indices = fam_data['QN']['index']

# for index_list in indices:
#     init, final = index_list[0], index_list[-1]
#     s = spos[init:final]
#     tousan.plot_scat_table(s, n_c=1, n_r=1)


In [9]:
# len(indices)

In [10]:
# ltime = pyaccel.lifetime.Lifetime(acc)
# tous_rate = ltime.touschek_data['rate']
# np.where(tous_rate <= 1e-6)[0].shape

In [11]:
fam_data = pymodels.si.get_family_data(acc)

In [12]:
spos[np.array(fam_data['B1']['index'])]


# BC, Q1, B1
s_test = [12.4387475, 6.8607475, 4.9667475, 7.3207475]

In [13]:
tousan.accep


(array([-0.04372908, -0.04372908, -0.04372908, ..., -0.04372908,
        -0.04372908, -0.04372908]),
 array([0.04372908, 0.04372908, 0.04372908, ..., 0.04372908, 0.04372908,
        0.04372908]))

In [14]:
tousan._accep

(array([-0.04372908, -0.04372908, -0.04372908, ..., -0.04372908,
        -0.04372908, -0.04372908]),
 array([0.04372908, 0.04372908, 0.04372908, ..., 0.04372908, 0.04372908,
        0.04372908]))

In [19]:
tousan.plot_normtousd(s_test)

In [16]:
spos[np.intp(pyaccel.lattice.find_indices(acc, 'fam_name', 'SFA1'))]

array([  7.3207475,  96.2072325, 110.9987275, 199.8852125, 214.6767075,
       303.5631925, 318.3546875, 407.2411725, 422.0326675, 510.9191525])

In [17]:
np.intp(pyaccel.lattice.find_indices(acc, 'fam_name', 'SFA1'))

array([  74, 1238, 1374, 2538, 2672, 3846, 3982, 5154, 5297, 6470])

In [27]:
array  = np.array([1,54,7,8,4,7,4,8,9,6,3,3,8,57,45,3,7756,8,34,5])
a = 8 

np.where(array == a)[0]

array([ 3,  7, 12, 17])